In [2]:
import pandas as pd

In [3]:
print("--- Starting Data Cleaning Process ---")
df = pd.read_csv('hotel_bookings.csv')

--- Starting Data Cleaning Process ---


In [4]:
print("--- Initial Data State ---")
print(f"Shape (Rows, Columns): {df.shape}")
print("\nMissing Values per Column:")
print(df.isnull().sum())
print(f"\nNumber of Duplicate Rows: {df.duplicated().sum()}")
df.info()
# --- Numerical Data Summary (to find outliers) ---
print("\nSummary Statistics for Numerical Columns:")
print(df.describe())


# Store initial stats for final comparison
initial_rows = df.shape[0]
initial_cols = df.shape[1]

--- Initial Data State ---
Shape (Rows, Columns): (119390, 32)

Missing Values per Column:
hotel                                  0
is_canceled                            0
lead_time                              0
arrival_date_year                      0
arrival_date_month                     0
arrival_date_week_number               0
arrival_date_day_of_month              0
stays_in_weekend_nights                0
stays_in_week_nights                   0
adults                                 0
children                               4
babies                                 0
meal                                   0
country                              488
market_segment                         0
distribution_channel                   0
is_repeated_guest                      0
previous_cancellations                 0
previous_bookings_not_canceled         0
reserved_room_type                     0
assigned_room_type                     0
booking_changes                        0
deposit

Key Data Quality Issues Found:

Missing Values:

company: Over 94% of values are missing. This likely indicates direct bookings, not missing data.

agent: Nearly 14% of values are missing, also likely indicating direct bookings.

country: A small number of rows (488) are missing the country code.

children: Four rows have missing values for the number of children.

Duplicate Data:

There are 31,994 complete duplicate rows in the dataset, which can significantly skew any analysis.

Incorrect Data Types:

children, agent, and company are stored as float types but should be integer types.

The reservation_status_date column is stored as a generic object (string) instead of a datetime object.

Structural Issues:

The arrival date is split across three separate columns (arrival_date_year, arrival_date_month, arrival_date_day_of_month), making date-based calculations difficult.

Invalid or Illogical Values:

There are bookings with a total of zero guests (adults, children, and babies are all 0).

At least one booking has a negative Average Daily Rate (ADR), which is impossible.

In [8]:
print("--- Handling Missing Values ---")

df['company'] = df['company'].fillna(0)
df['agent'] = df['agent'].fillna(0)
df['children'] = df['children'].fillna(0)

# Drop rows where 'country' is missing
df.dropna(subset=['country'], inplace=True)

# --- AFTER ---
print(f"Shape After: {df.shape}")
print("\nMissing Values After Handling (showing key columns):")
print(df[['country', 'agent', 'company', 'children']].isnull().sum())

--- Handling Missing Values ---
Shape After: (118902, 32)

Missing Values After Handling (showing key columns):
country     0
agent       0
company     0
children    0
dtype: int64


In [9]:
print("--- Removing Duplicates ---")
print(f"Shape Before: {df.shape}")
print(f"Number of Duplicates: {df.duplicated().sum()}")

# Remove duplicate rows
df.drop_duplicates(inplace=True)

# --- AFTER ---
print(f"Shape After: {df.shape}")
print(f"Number of Duplicates After: {df.duplicated().sum()}")

--- Removing Duplicates ---
Shape Before: (118902, 32)
Number of Duplicates: 31958
Shape After: (86944, 32)
Number of Duplicates After: 0


In [10]:
print("--- Correcting Data Types ---")
print("Data Types Before:")
print(df[['children', 'agent', 'company']].dtypes)

# Convert columns to integer
df[['children', 'agent', 'company']] = df[['children', 'agent', 'company']].astype(int)
# Convert reservation_status_date to datetime
df['reservation_status_date'] = pd.to_datetime(df['reservation_status_date'])

print("\nData Types After:")
print(df[['children', 'agent', 'company']].dtypes)

--- Correcting Data Types ---
Data Types Before:
children    float64
agent       float64
company     float64
dtype: object

Data Types After:
children    int64
agent       int64
company     int64
dtype: object


In [11]:
print("--- Creating Unified Date Column ---")
print("Columns Before:", [col for col in df.columns if 'date' in col])

# Create the unified 'arrival_date' column
df['arrival_date'] = pd.to_datetime(
    df['arrival_date_year'].astype(str) + '-' +
    df['arrival_date_month'].astype(str) + '-' +
    df['arrival_date_day_of_month'].astype(str),
    errors='coerce' # Handle potential conversion errors gracefully
)

# Drop the original date component columns
df.drop(['arrival_date_year', 'arrival_date_month', 'arrival_date_day_of_month'], axis=1, inplace=True)

print("Columns After:", [col for col in df.columns if 'date' in col])

--- Creating Unified Date Column ---
Columns Before: ['arrival_date_year', 'arrival_date_month', 'arrival_date_week_number', 'arrival_date_day_of_month', 'reservation_status_date']
Columns After: ['arrival_date_week_number', 'reservation_status_date', 'arrival_date']


In [12]:
print("--- Removing Invalid Data ---")
print(f"Shape Before: {df.shape}")

# Filter out bookings with zero guests
df = df[~((df['adults'] == 0) & (df['children'] == 0) & (df['babies'] == 0))]

# Filter out bookings with a negative ADR
df = df[df['adr'] >= 0]

# --- AFTER ---
print(f"Shape After: {df.shape}")

--- Removing Invalid Data ---
Shape Before: (86944, 30)
Shape After: (86782, 30)


In [13]:
# Feature Engineering
df['total_guests'] = df['adults'] + df['children'] + df['babies']
df['total_nights'] = df['stays_in_weekend_nights'] + df['stays_in_week_nights']

# Reset the DataFrame index
df.reset_index(drop=True, inplace=True)

In [14]:
# --- FINAL SUMMARY ---

print("--- Final Cleaning Summary ---")

print(f"Initial Shape: ({initial_rows}, {initial_cols})")

print(f"Final Shape: ({df.shape[0]}, {df.shape[1]})")

rows_removed = initial_rows - df.shape[0]

print(f"Total Rows Removed: {rows_removed} ({rows_removed / initial_rows:.2%})")

--- Final Cleaning Summary ---
Initial Shape: (119390, 32)
Final Shape: (86782, 32)
Total Rows Removed: 32608 (27.31%)


In [15]:
# Export the cleaned DataFrame

df.to_csv("hotel_bookings_cleaned.csv", index=False)



print("\nCleaned data exported successfully.")

print("\n---Data Cleaning Complete ---")


Cleaned data exported successfully.

---Data Cleaning Complete ---
